<a href="https://colab.research.google.com/github/ashish899999/LSTM-PROJECT/blob/main/fake_news_detection_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import pandas as pd

In [3]:
df=pd.read_csv(r'/content/fake_news_datset.csv')
df

,Unnamed: 0,label,news
0,0,0,us budget fight looms republicans flip fiscal ...
1,1,0,us military accept transgender recruits monday...
2,2,0,senior us republican senator let mr mueller jo...
3,3,0,fbi russia probe helped australian diplomat ti...
4,4,0,trump wants postal service charge much amazon ...
...,...,...,...
44893,44893,1,mcpain john mccain furious iran treated us sai...
44894,44894,1,justice yahoo settles email privacy classactio...
44895,44895,1,sunnistan us allied ‘ safe zone ’ plan take te...
44896,44896,1,blow 700 million al jazeera america finally ca...


In [31]:
df.sample()

,label,news
37727,1,department justice fines sheriff excluding ill...


In [7]:
!kaggle datasets download -d danielwillgeorge/glove6b100dtxt

Dataset URL: https://www.kaggle.com/datasets/danielwillgeorge/glove6b100dtxt
License(s): unknown
 98% 129M/131M [00:01<00:00, 75.6MB/s]
100% 131M/131M [00:01<00:00, 86.1MB/s]


In [9]:
import zipfile
zip_ref = zipfile.ZipFile('/content/glove6b100dtxt.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()

In [4]:
df=df.drop(['Unnamed: 0'],axis=1)

In [5]:
df.sample()

,label,news
38946,1,cowardly chain migration terrorist entered us ...


In [2]:
# import gensim.downloader as api

# # Load the pre-trained Word2Vec model from gensim's dataset
# word2vec_model = api.load("word2vec-google-news-300")

[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [10]:
import numpy as np

# Load the GloVe embeddings
embeddings_index = {}
with open('/content/glove.6B.100d.txt', encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


In [11]:
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import gensim.downloader as api
import numpy as np
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout

In [12]:
tokenizer=Tokenizer()
tokenizer.fit_on_texts(df['news'])
word_index=tokenizer.word_index
vocab_size=len(word_index)


In [13]:
vocab_size

229356

In [14]:
sequences = tokenizer.texts_to_sequences(df['news'])
padded_seq = pad_sequences(sequences, maxlen=500, padding='post', truncating='post')


In [15]:
# create embedding matrix

embedding_matrix = np.zeros((vocab_size+1, 100))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [16]:
embedding_matrix[1]


array([-0.15730999, -0.75502998,  0.36844999, -0.18957999, -0.16896001,
       -0.23157001, -0.22657999, -0.30186   ,  0.24372   ,  0.61896002,
        0.58995003,  0.047638  , -0.055164  , -0.70210999,  0.22084001,
       -0.69231999,  0.49419001,  1.42850006, -0.25362   ,  0.20031001,
       -0.26192001,  0.05315   , -0.048418  , -0.44982001,  0.54644001,
       -0.014645  , -0.015531  , -0.61197001, -0.91964   , -0.75279999,
        0.64842999,  1.0934    ,  0.052682  ,  0.33344999,  0.10532   ,
        0.59517002,  0.023104  , -0.37105   ,  0.29749   , -0.23683   ,
        0.079566  , -0.10326   ,  0.35885   , -0.28935   , -0.19881   ,
        0.22908001, -0.061435  ,  0.56127   , -0.017115  , -0.32868001,
       -0.78416997, -0.49375001,  0.34944001,  0.16278   , -0.061168  ,
       -1.31060004,  0.39151999,  0.124     , -0.20873   , -0.18472999,
       -0.56184   ,  0.55693001,  0.012114  , -0.54544997, -0.31409001,
        0.1       ,  0.31542999,  0.74756998, -0.47734001, -0.18

In [17]:
padded_seq[1]


array([     4,     82,   1264,   1331,  10046,    124,   1377,   1331,
            9,    593,     33,     32,  11650,      4,     82,   1585,
          124,   1135,     84,   1430,   1377,      2,    115,      8,
           17,      1,      3,     62,    605,   1464,   4623,   1689,
         1331,    352,     31,     84,   1412,   1430,     10,     91,
           10,    950,     28,     71,   1393,     62,      3,    671,
          166,    506,   1588,    925,     48,   1755,   3103,     82,
         1316,   3565,   1331,  10046,   1277,    417,    171,     86,
          126,      2,     62,   1184,   4623,      6,     86,    257,
          467,   3674,    854,   1558,    316,    461,    456,    566,
        35541,   5516,   1464,  11927,     62,    605,   1712,   9635,
            3,   1558,    342,   1113,      8,      3,   5734,   1056,
          626,     48,   7349,      5,    126,      2,    558,   1442,
         2131,    533,   1377,      2,    850,    901,    288,     75,
      

In [19]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(padded_seq, df['label'], test_size=0.20, random_state=42, stratify=df['label'])

In [20]:
from keras.layers import LSTM, Dropout, Dense, Embedding
from keras import Sequential

# model = Sequential([
#     Embedding(vocab_size+1, 100, weights=[embedding_matrix], trainable=False),
#     Dropout(0.2),
#     LSTM(128, return_sequences=True),
#     LSTM(128),
#     Dropout(0.2),
#     Dense(512),
#     Dropout(0.2),
#     Dense(256),
#     Dense(1, activation='sigmoid')
# ])

model = Sequential([
    Embedding(vocab_size+1, 100, weights=[embedding_matrix], trainable=False),
    Dropout(0.2),
    LSTM(128),
    Dropout(0.2),
    Dense(256),
    Dense(1, activation='sigmoid')
])

In [21]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics='accuracy')
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 100)         22935700  
                                                                 
 dropout (Dropout)           (None, None, 100)         0         
                                                                 
 lstm (LSTM)                 (None, 128)               117248    
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense (Dense)               (None, 256)               33024     
                                                                 
 dense_1 (Dense)             (None, 1)                 257       
                                                                 
Total params: 23086229 (88.07 MB)
Trainable params: 1505

In [22]:
history = model.fit(x_train, y_train, epochs=10, batch_size=256, validation_data=(x_test, y_test))


Epoch 1/10
141/141 [==============================] - 481s 3s/step - loss: 0.5898 - accuracy: 0.6766 - val_loss: 0.6918 - val_accuracy: 0.5255
Epoch 2/10
141/141 [==============================] - 505s 4s/step - loss: 0.6874 - accuracy: 0.5393 - val_loss: 0.6824 - val_accuracy: 0.5477
Epoch 3/10
141/141 [==============================] - 492s 4s/step - loss: 0.6832 - accuracy: 0.5479 - val_loss: 0.6802 - val_accuracy: 0.5609
Epoch 4/10
141/141 [==============================] - 509s 4s/step - loss: 0.6756 - accuracy: 0.5631 - val_loss: 0.6832 - val_accuracy: 0.5480
Epoch 5/10
141/141 [==============================] - 462s 3s/step - loss: 0.5964 - accuracy: 0.6569 - val_loss: 0.6394 - val_accuracy: 0.6514
Epoch 6/10
141/141 [==============================] - 488s 3s/step - loss: 0.6172 - accuracy: 0.6054 - val_loss: 0.5925 - val_accuracy: 0.6390
Epoch 7/10
141/141 [==============================] - 490s 3s/step - loss: 0.5482 - accuracy: 0.7098 - val_loss: 0.4301 - val_accuracy: 0.8329

In [34]:
# building a predictive system

def predict_sent(review):
  # tokenization and pad review
  sequence=tokenizer.texts_to_sequences([news])
  padded_sequence=pad_sequences(sequence,maxlen=200)
  prediction=model.predict(padded_sequence)
  sentiment="true news " if prediction[0][0]>0.5 else "fake news"
  return sentiment

# new_text = "This is a sample news article to be classified."
# max_length = max(len(seq) for seq in sequences)  # Use the same max_length as in training
# prepared_text = pad_sequences(new_text, tokenizer, max_length)

In [35]:
news="biden is a weak president , will lead to downfall of america"
sent=predict_sent(news)
print(sent)

1/1 [==============================] - 0s 102ms/step
fake news


In [27]:
news="donald trump is a presdient of india"
sent=predict_sent(news)
print(sent)

1/1 [==============================] - 0s 39ms/step
fake news


In [28]:
news="donald trump is a presdient of america"
sent=predict_sent(news)
print(sent)

1/1 [==============================] - 0s 90ms/step
fake news


In [37]:
news="us military accept transgender recruits monday"
sent=predict_sent(news)
print(sent)

1/1 [==============================] - 0s 91ms/step
fake news


In [36]:
news="department justice fines sheriff excluding illegals hiring wikipediathe immigration nationality act 1965 hart celler act abolished quota system based national origins american immigration policy since 1920s new law maintained percountry limits also created preference visa categories focused immigrants skills family relationships citizens us residents bill set numerical restrictions visas 170000 per year percountryoforigin quota however immediate relatives us citizens special immigrants restrictionsthe denver sheriff department accept penalty department justice federal probe found wrongly made us citizenship job requirement recent hiring spreethe sheriff department biggest sheriff office colorado pay 10000 fine required applications deputy sheriff jobs us citizens hiring beginning 2015 march 2016 department went hiring spree 200 deputies part ongoing reformthe department also go old applications find applicants eliminated citizenship status reconsider future jobsthe justice department made announcement monday saying sheriff department violated immigration nationality act without exemptionread"
sent=predict_sent(news)
print(sent)

1/1 [==============================] - 0s 46ms/step
fake news


In [ ]:
def preprocess_text(text):
    tokens = word_tokenize(text.lower())
    tokens = [word for word in tokens if word.isalpha() and word not in stop_words]
    return ' '.join(tokens)

In [39]:
def prepare_text(text, tokenizer, max_length):
    processed_text = preprocess_text(text)
    sequence = tokenizer.texts_to_sequences([processed_text])
    padded_sequence = pad_sequences(sequence, maxlen=max_length)
    return padded_sequence

In [40]:
# Function to predict fake news
def predict_fake_news(model, tokenizer, text, max_length):
    prepared_text = prepare_text(text, tokenizer, max_length)
    prediction = model.predict(prepared_text)
    predicted_label = (prediction > 0.5).astype(int)
    return 'Real' if predicted_label == 1 else 'Fake'

In [43]:
news="department justice fines sheriff excluding illegals hiring wikipediathe immigration nationality act 1965 hart celler act abolished quota system based national origins american immigration policy since 1920s new law maintained percountry limits also created preference visa categories focused immigrants skills family relationships citizens us residents bill set numerical restrictions visas 170000 per year percountryoforigin quota however immediate relatives us citizens special immigrants restrictionsthe denver sheriff department accept penalty department justice federal probe found wrongly made us citizenship job requirement recent hiring spreethe sheriff department biggest sheriff office colorado pay 10000 fine required applications deputy sheriff jobs us citizens hiring beginning 2015 march 2016 department went hiring spree 200 deputies part ongoing reformthe department also go old applications find applicants eliminated citizenship status reconsider future jobsthe justice department made announcement monday saying sheriff department violated immigration nationality act without exemptionread"
result = predict_fake_news(model, tokenizer, news, 100)
print(result)

NameError: name 'preprocess_text' is not defined